In [1]:
from mesa import Model
from mesa import Agent
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random
import numpy as np

from a_star_pathfinding import a_star

In [2]:
class AmongUs(Model):
    def __init__(self, map_name, n_crew, n_impo):
        super().__init__()

        # generate grid
        self.height = 138
        self.width = 242
        self.grid = MultiGrid(self.width, self.height, torus=True)

        # generate map
        self.generate_map(map_name)

        # create schedulers for agents
        self.schedule_Crewmate = RandomActivation(self)
        self.schedule_Imposter = RandomActivation(self)
        
        # initialize agents
        self.n_crew = n_crew
        self.n_impo = n_impo
        self.spawn_players()

    def generate_map(self, map_name):
        hard_walls = np.load(f'{map_name}/hardwalls.npy')
        
        for coord in hard_walls:
            coord[1] = self.height - 1 - coord[1]
            self.new_agent(Wall, tuple(coord))

    def spawn_players(self):
        # generate crewmates
        for _ in range(self.n_crew):
            self.new_agent(Crewmate, (125, 100))

        # generate imposters
        for _ in range(self.n_impo):
            self.new_agent(Imposter, (125, 100))

    def new_agent(self, agent_type, pos):
        '''
        Method that creates a new agent, and adds it to the correct scheduler.
        '''
        agent = agent_type(self.next_id(), self, pos)

        self.grid.place_agent(agent, pos)

        if agent_type == Crewmate or agent_type == Imposter:
            getattr(self, f'schedule_{agent_type.__name__}').add(agent)

        
    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''

        self.schedule_Crewmate.step()
        self.schedule_Imposter.step()

        # self.datacollector.collect(self)

    def play_match(self, step_count=50):
        '''
        Method that runs the model for a specific amount of steps.
        '''
        for i in range(step_count):
            self.step()


In [3]:
class Crewmate(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goal = (90, 40)

    def find_goal(self):
        self.path = a_star(self.pos, self.goal, self.model.grid)

    def step(self):
        if self.path == []:
            self.find_goal()
        
        grid = self.model.grid
        neighborhood = grid.get_neighborhood(self.pos, True)

        neighbors = self.model.grid.get_neighbors(self.pos, True)

        if len(neighbors) > 0:
            for neighbor in neighbors:
                if type(neighbor) == Wall and neighbor.pos in neighborhood:
                    neighborhood.remove(neighbor.pos)

        # random movement
        # grid.move_agent(self, random.choice(neighborhood))

        if self.path != []:
            grid.move_agent(self, self.path.pop())



class Imposter(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos

    def step(self):
        grid = self.model.grid
        neighborhood = grid.get_neighborhood(self.pos, True)

        neighbors = self.model.grid.get_neighbors(self.pos, True)

        if len(neighbors) > 0:
            for neighbor in neighbors:
                if type(neighbor) == Wall and neighbor.pos in neighborhood:
                    neighborhood.remove(neighbor.pos)

        grid.move_agent(self, random.choice(neighborhood))

class Wall(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Obstruction(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

In [4]:
amongUs = AmongUs('the_skeld', 3, 1)

amongUs.play_match()

NameError: name 'A_star' is not defined